<a href="https://colab.research.google.com/github/JasonManesis/Vehicle-Navigation-Shortest-Path/blob/main/Vehicle_Navigation_Shortest_Path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
.....................................................SOLUTION SUBMISSION...............................................................

The type of project we worked on for our summative is that we have created our own smaller and simpler version of mapping platform
(similar with Google maps), which is targeted for international students (even locals) in Kigali ,primarily at ALU to help them navigate the 
short pathes from source (a specific starting point) to destination (to the place they want to go).

 """

'\n.....................................................SOLUTION SUBMISSION...............................................................\n\nThe type of project we worked on for our summative is that we have created our own smaller and simpler version of mapping platform\n(similar with Google maps), which is targeted for international students (even locals) in Kigali ,primarily at ALU to help them navigate the \nshort pathes from source (a specific starting point) to destination (to the place they want to go).\n\n '

In [2]:
pip install osmnx==0.16.2

     |████████████████████████████████| 87 kB 4.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 1.0 MB 53.4 MB/s 
     |████████████████████████████████| 1.0 MB 47.0 MB/s 
     |████████████████████████████████| 6.3 MB 42.0 MB/s 
     |████████████████████████████████| 11.2 MB 46.8 MB/s 
     |████████████████████████████████| 16.7 MB 42.9 MB/s 
     |████████████████████████████████| 900 kB 41.4 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requ

In [3]:
pip install contextily 

     |████████████████████████████████| 19.3 MB 69.1 MB/s 


In [4]:
# Here we have Imported all the necessary libraries and dependencies that are crutial for our program to run.
import numpy as np
import math
import folium
import contextily as ctx
import shapely
import osmnx as ox
import geopandas as gpd
from pyproj import Transformer
from shapely.geometry import  Point, Polygon, LineString
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Circle
from osgeo import ogr
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Here we have created a function called pointproject which will enable us to project/print out the actual location of our source and destination points.
def pointreproject(point_x,point_y,inp_CRS,out_CRS):
  
  """ 
  Purpose of the function:
      We will be converting the values from (latitude, longitude) to native map projection (x,y) coordinates and vice versa. 

        Args:
            point_x: input CRS coordinate (x or latitude). 
            point_y: input CRS coordinate (y or longitude).
            inp_CRS: reference system of input coordinates: 'epsg:4326','epsg:3857'... 
            out_CRS: reference system of output coordinates: 'epsg:4326','epsg:3857'...

        Returns:
            A tuple (latitude, longitude) or (x,y) of the reprojected coordinates of the input point in the output CRS.

            NB: EPSG 3857 and EPSG 4326 are just a type of coordinate systems.
              
  """
  # This will convert/transform the reference system of the coordinates into x,y or longitude, latitude and vice versa.
  transformer = Transformer.from_crs(inp_CRS, out_CRS)
  out_x,out_y = transformer.transform(point_x, point_y)
  transformer = []
  # Returns the transformed out put
  return out_x,out_y

In [6]:

def rgraphfrompoints(starting_point,ending_point):

  """
  Purpose of the function:
   -->To create a graph, with edges (roads) and nodes (road intersections), based on the two given points.
        The two given points must be in the WGS84 coordinate reference system and in the form of (latitude, longitude).  
        NB: WGS84 is also a coordinate reference system which adds Greenwich as a starting point (Prime meridian) for longtiude (0 degree).

        Args:
            starting_point: the route's starting point. -> A tuple of (latitude, longitude).
            ending_point: the route's ending point. -> A tuple of (latitude, longitude).

        Returns:
            A reprojected (from EPSG:4326 to EPSG:3857) MultiDiGraph from the osmnx package.
              
  """
  #Here we input points of our geographical coordinates.  
  lat1 = starting_point[0]
  lat2 = ending_point[0]
  long1 = starting_point[1]
  long2 = ending_point[1]
  
  #Here we will find the function's argument values.
  north = lat2
  south = lat1
  if  lat1 > lat2:
    north = lat1 
    south = lat2

  east = long2
  west = long1
  if  long1 > long2:
    east = long1 
    west = long2

  #Here we will construct the road graph using the osmnx package
  roadgraph = ox.graph_from_bbox(north+0.01, south-0.01, east+0.01, west-0.01, network_type = 'drive', simplify=False )

  #Here we will reproject the road graph.
  projected_roadgraph = ox.projection.project_graph(roadgraph, to_crs='epsg:3857')
# ANd return the reprojected graph
  return projected_roadgraph 

In [7]:
def nearest_edge_node_id(point,roadgraph):

  """
  Purpose of the function:
      --> To find the nearest network's node id for the given point, which lies in the nearest network edge.
      Input Point -> Nearest Edge Point -> Nearest Network Node (Route Starting/Ending Point).
      The given point's coordinates must be in the EPSG:4326.  

      Args:
          point: The point:(latitude, longitude) that needs to be transfered to the nearest network node.
          roadgraph: A reprojected (from EPSG:4326 to EPSG:3857) MultiDiGraph from the osmnx package.

      Returns:
          The nearest network's node id for the given point, which is in the nearest network edge. 
      
  """
  #Here we will reproject point coordinates from EPSG:4326 to the EPSG:3857.   
  point_x, point_y = pointreproject(point[0], point[1],'epsg:4326','epsg:3857')

  #Here we will find the nearest road.
  _, _, _, nearest_road = ox.distance.get_nearest_edge(roadgraph, (point_y,point_x), return_geom=True)
  
  #Here we find all point coordinates that are constructing the specific road that we want.
  nearest_road_coords = np.array(nearest_road.xy)

  #Here we will calculate all distances between the input point and all the points constructing the nearest road line.
  nearest_distances = []
  for i in range(0,nearest_road_coords[0].size):
    e_dist = ox.distance.euclidean_dist_vec(point_y,point_x,nearest_road_coords[1][i],nearest_road_coords[0][i])
    nearest_distances.append(e_dist)
  mindist = min(nearest_distances)
  for i in range(0,len(nearest_distances)):
    if nearest_distances[i]==mindist: pos=i
                                
  #Here we will assign the nearest edge node coordinates.
  nearest_edge_node_x = nearest_road_coords[0][pos]  
  nearest_edge_node_y = nearest_road_coords[1][pos]  

  #Here we will find the nearest network's node id, which has the minimum distance from the road point that has the minimum distance from the input point.
  node_id= ox.distance.get_nearest_node(roadgraph, (nearest_edge_node_y,nearest_edge_node_x),  method='euclidean')

# At last we will return the node id.
  return node_id

In [8]:
def pointreproject2edge(point,roadgraph):

  """
     Purpose of the function:
    --> To Find the nearest network's edge for the given point, and reprojects it perpendicularly to its nearest edge.
        If the orthogonal projection of the point lies outside the nearest edge's nodes, then the point isn't reprojected
        but it is transferred to the nearest network node instead.

        The given point's coordinates must be in the EPSG:4326, and following the form of (latitude, longitude).  

        Args:
            point: The point:(latitude, longitude) that needs to be projected to the nearest network edge.
            roadgraph: A MultiDiGraph (EPSG:3857) from the osmnx package.

        Returns:
            A tuple (latitude, longitude) of the reprojected coordinates of the input point.
        
  """

  #Here we will reproject point coordinates from EPSG:4326 to the EPSG:3857.   
  point_x, point_y = pointreproject(point[0], point[1],'epsg:4326','epsg:3857')

  #Here we will find the nearest road.
  _, _, _, nearest_road, dist__pp = ox.distance.get_nearest_edge(roadgraph, (point_y,point_x), return_geom=True, return_dist=True)

  #Here we will extract the nearest road coordinates.
  nearest_road_coords = np.array(nearest_road.xy)

  #We will calculate the euclidean distance between the input point and the nearest edge's first node.
  dist__pv = ox.distance.euclidean_dist_vec(point_y,point_x,nearest_road_coords[1][0],nearest_road_coords[0][0])

  #This will calculate the hypotenuse.
  dist = np.sqrt(dist__pv**2-dist__pp**2)

  #Here we will create the corresponding circle.
  circle = Point(nearest_road_coords[0][0], nearest_road_coords[1][0]).buffer(dist).boundary

  #Then we will find the circle's intersection point with the given edge.
  intersection_points = circle.intersection(nearest_road)
  
  # set the bool to False 
  lies_in = False

  #If the orthogonal projection of the point lies inside the nearest edge's nodes.
  if type(intersection_points) == shapely.geometry.point.Point:
    lies_in = True
    intersection_point_x = intersection_points.x
    intersection_point_y = intersection_points.y
 
  #If the orthogonal projection of the point lies outside the nearest edge's nodes.
  else:
    gdf = ox.utils_graph.graph_to_gdfs(roadgraph)
    nodes_gdf = gdf[0].reset_index(drop=True) 
    nearest_node_id = nearest_edge_node_id(point,roadgraph)
    intersection_point_x = nodes_gdf.loc[(nodes_gdf['osmid']==nearest_node_id)].x.values[0]
    intersection_point_y = nodes_gdf.loc[(nodes_gdf['osmid']==nearest_node_id)].y.values[0]

  #Then we will find the nearest node id from the intersection point.
  node_id = ox.distance.get_nearest_node(roadgraph, (intersection_point_y,intersection_point_x),  method='euclidean')

  #Finally we will reproject the calculated grid point from epsg:3857 to epsg:4326.
  point_lat, point_long = pointreproject(intersection_point_x, intersection_point_y,'epsg:3857','epsg:4326')
  
  return point_lat, point_long, lies_in

In [9]:
def routeplot(start,end):
    
    """
    Purpose of the function:
      --> To find and plot the shortest route based on Dijkstra's pathfinding algorithm- between the 
      two given points in the given roadgraph.The given points coordinates must be in the EPSG:4326 
      and they must be following the form of (latitude, longitude). 

      Args:
          starting_point: the route's starting point. -> A tuple of (latitude, longitude).
          ending_point: the route's ending point. -> A tuple of (latitude, longitude).

      Returns:
          A Folium map with the given starting and ending points and the shortest path between them. 
    
  """

  # Here we set some specific points/areas where we would like to have a possible shortest paths in between and entered the actual coordinates of the locations.
  # We set start for our starting points or places
  
    if start == "ALU Campus": 
          starting_point = (-1.9295149476454851, 30.15216891903913)   #-1.9295149476454851, 30.15216891903913
    elif start == "Remera":
          starting_point = (-1.9603797582432974, 30.10894093684139)   # -1.9603797582432974, 30.10894093684139
    elif start == "Down Town":
          starting_point = (-1.9434555360830934, 30.05733382342882)   #-1.9434555360830934, 30.05733382342882
    elif start == "Kimronko":
          starting_point = (-1.9494401595403812, 30.125003458771193)   #-1.9494401595403812, 30.125003458771193
          
    # We have sent end to our final destinations
    if end == "ALU Campus":
          ending_point = (-1.9295149476454851, 30.15216891903913)
    elif end == "Remera":
          ending_point = (-1.9603797582432974, 30.10894093684139) 
    elif end == "Down Town":
          ending_point = (-1.9434555360830934, 30.05733382342882)
    elif end == "Kimronko":
          ending_point = (-1.9494401595403812, 30.125003458771193)  
    
    #We will calculate the roadgraph between the starting and ending points by using rgraphfrompoints function found above .
    roadgraph = rgraphfrompoints(starting_point,ending_point)

    #Then we will find the routes starting end ending point which lies inside the given roadgraph.
    p_lat1,p_long1,lies_in1 = pointreproject2edge(starting_point,roadgraph)
    p_lat2,p_long2,lies_in2 = pointreproject2edge(ending_point,roadgraph)

    #We will find the nearest edge node id next.
    node1_id = nearest_edge_node_id(starting_point,roadgraph)
    node2_id = nearest_edge_node_id(ending_point,roadgraph)
    
    #Then we will calculate the sortest path between the two given nodes.
    nodeids = ox.shortest_path(roadgraph, node1_id, node2_id)

    #Here we will create the necessary geodataframes.
    gdf = ox.utils_graph.graph_to_gdfs(roadgraph)
    nodes_gdf = gdf[0].reset_index(drop=True) 

    #we will create our Map's center point.
    center_lat = (starting_point[0]+ending_point[0])/2
    center_long = (starting_point[1]+ending_point[1])/2
    center = [center_lat,center_long]

    #The next is where we have defined the map.
    map = folium.Map(location=center, zoom_start=13, crs='EPSG3857')

    #Then we will Plot Original points using folium package dependencies we have installed earlier on.
    folium.Marker((starting_point[0], starting_point[1]), 
                  popup="Original Starting Point", tooltip="Original Starting Point", 
                  icon=folium.Icon(color='pink', icon="fas fa-car", prefix='fa')).add_to(map);

    folium.Marker((ending_point[0], ending_point[1]), 
                  popup="Original Destination", tooltip="Original Destination", 
                  icon=folium.Icon(color='pink', icon="fas fa-crosshairs", prefix='fa')).add_to(map);

    #First we will Plot Starting and Destination points.
    folium.Marker((p_lat1,p_long1), 
                  popup="Starting Point", tooltip="Starting Point", 
                  icon=folium.Icon(color='darkblue', icon="fas fa-car", prefix='fa')).add_to(map);

    folium.Marker((p_lat2,p_long2), 
                  popup="Destination", tooltip="Destination", 
                  icon=folium.Icon(color='darkblue', icon="fas fa-crosshairs", prefix='fa')).add_to(map);

    #Here we will create a list of tuples (for the locations) for each of route's node coordinates.
    locations = []
    locations.append((p_lat1,p_long1)) #Add the starting point to the route.
    for node_id in nodeids:
      p_lat = nodes_gdf.loc[(nodes_gdf['osmid']==node_id)].lat.values[0]
      p_long = nodes_gdf.loc[(nodes_gdf['osmid']==node_id)].lon.values[0]
      locations.append((p_lat,p_long))
    locations.append((p_lat2,p_long2)) #Add the ending point to the route.

    #Then we will cut the final route segment if the starting point lies in it.
    if lies_in1: 
      r1 = pointreproject(locations[1][0],locations[1][1],'epsg:4326','epsg:3857')
      r2 = pointreproject(locations[2][0],locations[2][1],'epsg:4326','epsg:3857')
      road1 = LineString([r1,r2])
      point1_x, point1_y = pointreproject(p_lat1,p_long1,'epsg:4326','epsg:3857')
      point1 = Point(point1_x, point1_y)
      if road1.distance(point1) < 1e-8 :del locations[1] 

    #Here we will Cut the final route segment if the ending point lies in it.

    """
     --> Here some users might select the same point as their starting and destination point.
      When this happens it will generate an index error. Just to make sure this wont happen, We have used the try and except error handling
      (Which is also more like our test case in this situation) the program wont crash and will notify the user to enter a diffrent end point.

    """

    try:
      if lies_in2: 
        r3 = pointreproject(locations[-3][0],locations[-3][1],'epsg:4326','epsg:3857')
        r4 = pointreproject(locations[-2][0],locations[-2][1],'epsg:4326','epsg:3857')
        road2 = LineString([r3,r4])
        point2_x, point2_y = pointreproject(p_lat2,p_long2,'epsg:4326','epsg:3857')
        point2 = Point(point2_x, point2_y)
        if road2.distance(point2) < 1e-8 :del locations[-2] 
    except IndexError:
      print('Please choose a different destination point.')


    #We will create a folium polyline from the list of "locations".
    route = folium.PolyLine(locations=locations,tooltip='Route', smooth_factor=0.3, color="purple", weight=2.5, opacity=1)

    #Now after creating the folium polyline we will Plot the route.
    route.add_to(map);

    #We will Save the map to html file.
    map.save('route.html') #saves the .html file in the local directory.

    #Finally Plot the map.
    return map

In [ ]:
#@title Xplore Kigali
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

Origin = "ALU Campus" #@param ["ALU Campus", "Down Town", "Kimronko", "Remera"]
Destination = "ALU Campus" #@param ["ALU Campus", "Down Town", "Kimronko", "Remera"]

routeplot(Origin, Destination)


Please choose a different destination point.
